You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an presentation

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [6]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [7]:
import os
# importando bibliotecas necessárias
from dotenv import load_dotenv
import os
# from groq import Groq
import re

# carregando arquivo de variáveis de ambiente com chave da api
load_dotenv()
print("importação funcionou")

os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


importação funcionou


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [8]:
planner = Agent(
    role="Planejador de conteúdo",
    goal="Planeje conteúdo envolvente e factualmente preciso sobre o tópico: {topic}",
    backstory="Você está trabalhando no planejamento de uma apresentação em uma feira de eventos de agroindústria "
    "sobre o tópico: {topic}."
    "Você coleta informações que ajudam o "
    "público a aprender algo"
    "e a tomar decisões informadas. "
    "Seu trabalho é a base para o "
    "Redator de Conteúdo escrever uma apresentação sobre este tópico",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Redator de Conteúdo",
    goal="Escreva uma apresentação no formato power point ou de marcadores, perspicaz e factualmente preciso sobre o tópico: {topic}",
    backstory="Você está trabalhando em uma redação "
    "de uma apresentação sobre tópico: {topic}. "
    "Você baseia sua redação no trabalho do "
    "Planejador de Conteúdo, que fornece um esboço "
    "e contexto relevante sobre o tópico. "
    "Você segue os principais objetivos e "
    "a direção do esboço, "
    "conforme fornecido pelo Planejador de Conteúdo. "
    "Você também fornece insights objetivos e imparciais "
    "e os respalda com informações "
    "fornecidas pelo Planejador de Conteúdo.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edite uma determinada apresentação para alinhá-la com o "
    "estilo de escrita da organização do evento. Trata-se da Tecnoshow que acontece em Rio Verde, Goiás",
    backstory="Você é um editor que recebe uma apresentação "
    "do Redator de Conteúdo. "
    "Seu objetivo é revisar a apresentação "
    "para garantir que ela siga as melhores práticas que uma apresentação para a Tecnoshow que acontece em Rio Verde em Goiás possa ter."
    "forneça pontos de vista equilibrados com referências científicas, "
    "ao fornecer opiniões ou afirmações, "
    "e também evite grandes tópicos controversos "
    "ou opiniões quando possível.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Priorize as últimas publicações científicas, "
        "e notícias importantes sobre o tópico: {topic}.\n"
        "2. Identifique o público-alvo da Tecnoshow, considerando "
        "seus interesses e pontos fracos.\n"
        "3. Desenvolva um esboço de conteúdo detalhado, incluindo "
        "uma introdução, pontos-chave, problemas, soluções e uma chamada para ação.\n"
        "4. Inclua palavras-chave e exemplos relevantes de mudanças climáticas e sobre o momento do plantio ocorrido na região sul de Goiás."),
    expected_output="Um documento de plano de conteúdo abrangente.",
    agent=planner,
)

### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use o plano de conteúdo para elaborar uma "
        "apresentação atraente sobre o tópico: {topic}.\n"
        "2. Incorpore palavras-chave, problemas reais da região sul de Goiás que envolvem o tópico.\n"
        "3. As seções/subtítulos são nomeados corretamente "
        "de forma envolvente.\n"
        "4. Garanta que a apresentação seja estruturada com uma "
        "introdução envolvente, um corpo perspicaz "
        "e uma conclusão resumida.\n"
        "5. Revise os erros gramaticais.\n"),
    expected_output="Uma apresentação acadêmica bem escrita "
        "em formato markdown ou powerpoint, pronta ser apresentada, "
        "cada seção deve ter 1 ou 2 parágrafos.",
    agent=writer,
)

### Task: Edit

In [13]:
edit = Task(
    description=("Revise a apresentação, para a feira tecnoshow, fornecida para "
                    "erros gramaticais e "
                    "alinhamento com a voz da marca."),
    expected_output="Uma apresentação bem escrita em formato markdown ou powerpoint, "
                    "pronta para ser apresentada na feira Tecnoshow, "
                    "cada seção deve ter 1 ou 2 parágrafos.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
result = crew.kickoff(inputs={"topic": "Aplicação de inteligência artificial na previsão climática e na gestão do momento do plantio"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4c6a3628-1b60-4fa4-b1cf-d97ee259d6bb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
       Status: Executing Task...
    └── 🤖 Agent: Planejador de conteúdo
            Status: In Progress

# Agent: Planejador de conteúdo
## Task: 1. Priorize as últimas publicações científicas, e notícias importantes sobre o tópico: Aplicação de inteligência artificial na previsão climática e na gestão do momento do plantio.
2. Identifique o público-alvo da Tecnoshow, considerando seus interesses e pontos fracos.
3. Desenvolva um esboço de conteúdo detalhado, incluindo uma introdução, pontos-chave, problemas, soluções e uma chamada para ação.
4. Inclua palavras-chave e exemplos relevantes de mudanças climáticas e sobre o momento do plantio ocorrido na região sul de Goiás.


🚀 Crew: crew
└── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
       Status: Executing Task...
    └── 🤖 Agent: Planejador de conteúdo
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Editor
## Final Answer: 
# **Aplicação de Inteligência Artificial na Previsão Climática e na Gestão do Momento do Plantio**

## **Introdução:**
A modernização agrícola tem sido impulsionada pela utilização da inteligência artificial na previsão climática e no gerenciamento do plantio. A capacidade de antecipar com precisão as condições climáticas e identificar o momento ideal para plantar impacta diretamente na eficiência e produtividade do setor agrícola em Rio Verde, Goiás.

## **Pontos-Chave:**
1. **Inteligência Artificial na Previsão Climática:**
    - A inteligência artificial faz uso de dados históricos, modelos preditivos e machine learning para oferecer previsões climáticas mais acuradas.
    - A precisão na previsão do clima é crucial para auxiliar os agricultores na tomada de decisões estratégicas durante o plantio.
    - Tecnologias como sensores, drones e softwares de monitoramento desempenham um papel fundamental na coleta de dados e no aprimoramento das previsõ

🚀 Crew: crew
├── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
│      Assigned to: Planejador de conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Planejador de conteúdo
│           Status: ✅ Completed
├── 📋 Task: 72fbebd6-f89f-4d1f-a4e9-fa72dc72f2ca
│      Assigned to: Redator de Conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Redator de Conteúdo
│           Status: ✅ Completed
└── 📋 Task: 4ddd5041-96bd-4b0c-a4dc-a4c4737d0cdf
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
│      Assigned to: Planejador de conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Planejador de conteúdo
│           Status: ✅ Completed
├── 📋 Task: 72fbebd6-f89f-4d1f-a4e9-fa72dc72f2ca
│      Assigned to: Redator de Conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Redator de Conteúdo
│           Status: ✅ Completed
└── 📋 Task: 4ddd5041-96bd-4b0c-a4dc-a4c4737d0cdf
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4ddd5041-96bd-4b0c-a4dc-a4c4737d0cdf                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4c6a3628-1b60-4fa4-b1cf-d97ee259d6bb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [21]:
result.raw

'**Apresentação: Como Pescar no Rio Araguaia**\n\n**Introdução:**\nSejam todos bem-vindos à nossa apresentação "Como Pescar no Rio Araguaia". No decorrer deste evento, nosso objetivo é oferecer orientações valiosas para os pescadores que buscam explorar as águas do rio Araguaia. Abordaremos técnicas de pesca, equipamentos essenciais, práticas sustentáveis e regulamentações locais, fornecendo um guia completo para uma pesca responsável e gratificante.\n\n**Importância da Pesca no Rio Araguaia**\nA pesca no rio Araguaia desempenha um papel crucial na economia local e no turismo, atraindo entusiastas da pesca de todo o Brasil em busca de paisagens deslumbrantes e experiências únicas.\n\n**Técnicas de Pesca**\nExploraremos diversas técnicas comumente utilizadas no rio Araguaia, como pesca de arremesso, pesca de fundo e pesca de corrico. Compartilharemos conhecimentos essenciais para que os pescadores alcancem o sucesso em suas pescarias na região.\n\n**Equipamentos Necessários**\nDiscutire

- Display the results of your execution as markdown in the notebook.

In [23]:
from IPython.display import Markdown
Markdown(result.raw)

**Apresentação: Como Pescar no Rio Araguaia**

**Introdução:**
Sejam todos bem-vindos à nossa apresentação "Como Pescar no Rio Araguaia". No decorrer deste evento, nosso objetivo é oferecer orientações valiosas para os pescadores que buscam explorar as águas do rio Araguaia. Abordaremos técnicas de pesca, equipamentos essenciais, práticas sustentáveis e regulamentações locais, fornecendo um guia completo para uma pesca responsável e gratificante.

**Importância da Pesca no Rio Araguaia**
A pesca no rio Araguaia desempenha um papel crucial na economia local e no turismo, atraindo entusiastas da pesca de todo o Brasil em busca de paisagens deslumbrantes e experiências únicas.

**Técnicas de Pesca**
Exploraremos diversas técnicas comumente utilizadas no rio Araguaia, como pesca de arremesso, pesca de fundo e pesca de corrico. Compartilharemos conhecimentos essenciais para que os pescadores alcancem o sucesso em suas pescarias na região.

**Equipamentos Necessários**
Discutiremos os equipamentos adequados para a pesca no rio Araguaia, desde varas de pesca específicas até iscas e anzóis. Preparando os pescadores com o equipamento correto para enfrentar os desafios que o rio apresenta.

**Práticas de Pesca Sustentável**
Enfatizaremos a importância da pesca sustentável para preservar os recursos naturais do rio Araguaia, destacando a necessidade de uma abordagem consciente para garantir a sustentabilidade a longo prazo da atividade pesqueira.

**Regulamentações Locais**
Informaremos sobre as leis e regulamentos locais relacionados à pesca no rio Araguaia, incluindo tamanhos mínimos de peixes, cotas de captura e outras diretrizes importantes que os pescadores devem respeitar para uma pesca responsável.

**Chamada para Ação**
Encorajaremos os pescadores a praticarem a pesca responsável, respeitando as normas locais, contribuindo para a preservação do rio Araguaia e desfrutando de uma experiência de pesca gratificante e sustentável.

**Palavras-Chave:**
Pesca no Rio Araguaia, Técnicas de Pesca, Equipamentos de Pesca, Pesca Sustentável, Regulamentações Locais, Preservação Ambiental.

**Impacto das Mudanças Climáticas na Região Sul de Goiás:**
Iremos abordar os impactos das mudanças climáticas na fauna aquática do rio Araguaia, assim como as adaptações necessárias na pesca para lidar com essas alterações. Exploraremos também as variações no ciclo de reprodução de peixes e como os pescadores podem se adaptar para garantir a continuidade da atividade pesqueira.

Esta apresentação tem como objetivo capacitar os pescadores da Tecnoshow a aproveitarem ao máximo suas pescarias no rio Araguaia, promovendo a conscientização sobre a importância da sustentabilidade e preservação dos recursos naturais. Esperamos que aproveitem e tenham ótimas pescarias!

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [24]:
topic = "Aplicação de inteligência artificial na previsão climática e na gestão do momento do plantio"
result = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4c6a3628-1b60-4fa4-b1cf-d97ee259d6bb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
       Status: Executing Task...
    └── 🤖 Agent: Planejador de conteúdo
            Status: In Progress

# Agent: Planejador de conteúdo
## Task: 1. Priorize as últimas publicações científicas, e notícias importantes sobre o tópico: Aplicação de inteligência artificial na previsão climática e na gestão do momento do plantio.
2. Identifique o público-alvo da Tecnoshow, considerando seus interesses e pontos fracos.
3. Desenvolva um esboço de conteúdo detalhado, incluindo uma introdução, pontos-chave, problemas, soluções e uma chamada para ação.
4. Inclua palavras-chave e exemplos relevantes de mudanças climáticas e sobre o momento do plantio ocorrido na região sul de Goiás.


🚀 Crew: crew
└── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
       Status: Executing Task...
    └── 🤖 Agent: Planejador de conteúdo
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Editor
## Final Answer: 
# Aplicação de inteligência artificial na previsão climática e na gestão do momento do plantio

## Introdução:
Na indústria agroindustrial, a aplicação da inteligência artificial na previsão climática e na gestão do momento do plantio tem se mostrado crucial para aumentar a eficiência e a produtividade no setor agrícola. A análise de dados climáticos em tempo real e a previsão de padrões climáticos por meio da inteligência artificial oferecem insights valiosos para os agricultores, possibilitando uma tomada de decisão mais informada e precisa. Nesta apresentação, vamos explorar como a integração da inteligência artificial tem impactado positivamente a otimização do momento do plantio e a mitigação dos riscos climáticos na região sul de Goiás.

## Pontos-Chave:
1. **Importância da previsão climática na agricultura:** A precisão na previsão do clima é essencial para o sucesso das safras.
   
2. **Como a inteligência artificial auxilia na previsão climá

🚀 Crew: crew
├── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
│      Assigned to: Planejador de conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Planejador de conteúdo
│           Status: ✅ Completed
├── 📋 Task: 72fbebd6-f89f-4d1f-a4e9-fa72dc72f2ca
│      Assigned to: Redator de Conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Redator de Conteúdo
│           Status: ✅ Completed
└── 📋 Task: 4ddd5041-96bd-4b0c-a4dc-a4c4737d0cdf
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 56f9385e-6b01-41ff-aa12-04ff5304fc50
│      Assigned to: Planejador de conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Planejador de conteúdo
│           Status: ✅ Completed
├── 📋 Task: 72fbebd6-f89f-4d1f-a4e9-fa72dc72f2ca
│      Assigned to: Redator de Conteúdo
│      Status: ✅ Completed
│   └── 🤖 Agent: Redator de Conteúdo
│           Status: ✅ Completed
└── 📋 Task: 4ddd5041-96bd-4b0c-a4dc-a4c4737d0cdf
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4ddd5041-96bd-4b0c-a4dc-a4c4737d0cdf                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4c6a3628-1b60-4fa4-b1cf-d97ee259d6bb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [25]:
Markdown(result.raw)

# Aplicação de inteligência artificial na previsão climática e na gestão do momento do plantio

## Introdução:
Na indústria agroindustrial, a aplicação da inteligência artificial na previsão climática e na gestão do momento do plantio tem se mostrado crucial para aumentar a eficiência e a produtividade no setor agrícola. A análise de dados climáticos em tempo real e a previsão de padrões climáticos por meio da inteligência artificial oferecem insights valiosos para os agricultores, possibilitando uma tomada de decisão mais informada e precisa. Nesta apresentação, vamos explorar como a integração da inteligência artificial tem impactado positivamente a otimização do momento do plantio e a mitigação dos riscos climáticos na região sul de Goiás.

## Pontos-Chave:
1. **Importância da previsão climática na agricultura:** A precisão na previsão do clima é essencial para o sucesso das safras.
   
2. **Como a inteligência artificial auxilia na previsão climática:** Algoritmos de IA analisam grandes volumes de dados para fornecer previsões mais precisas.
   
3. **Gestão do momento do plantio e sua relevância para a produtividade agrícola:** O momento correto do plantio é fundamental para o desenvolvimento saudável das culturas e para a maximização da produção.
   
4. **Benefícios da integração da inteligência artificial na gestão do momento do plantio:** A IA permite recomendações mais assertivas e redução de perdas.
   
5. **Estudos de caso de sucesso na região sul de Goiás:** Exemplos concretos de como a inteligência artificial tem contribuído para a agricultura na região.

## Problemas:
- Incertezas climáticas impactando o momento ideal de plantio.
- Prejuízos causados por decisões equivocadas na gestão do plantio.
- Falta de ferramentas eficazes para prever com precisão eventos climáticos extremos.

## Soluções:
- Utilização de algoritmos de inteligência artificial para análise preditiva do clima.
- Softwares especializados na recomendação do melhor momento de plantio.
- Integração de sensores IoT para coleta de dados climáticos em tempo real.
- Implementação de sistemas de monitoramento automatizados.

## Chamada para Ação:
Diante do cenário desafiador da agricultura moderna, é fundamental que os produtores adotem soluções inovadoras, como a inteligência artificial, para aprimorar suas práticas de gestão do plantio. Ao investir em tecnologias que permitem uma previsão mais precisa do clima e uma melhor gestão do momento do plantio, os agricultores podem aumentar sua produtividade, reduzir riscos e contribuir para o desenvolvimento sustentável do setor agrícola.

**Palavras-chave:** inteligência artificial, previsão climática, gestão do momento do plantio, agricultura de precisão, região sul de Goiás.

Exemplos relevantes de mudanças climáticas e do momento do plantio na região sul de Goiás podem incluir eventos de seca extrema afetando a produção agrícola e a importância de identificar o período ideal de plantio para culturas específicas, como soja e milho, levando em consideração as condições climáticas locais.

A adoção de práticas baseadas em inteligência artificial pode colocar os agricultores em vantagem na gestão agrícola, garantindo melhores colheitas e maior sustentabilidade.

Agradeço pela atenção!